In [7]:
import numpy as np

In [8]:
class Gram_Schmidth_Process:
    def __init__(self, matrix):
        self.matrix = matrix
        self.num_rows, self.num_columns = self.GET_MATRIX_ROWS_COLUMNS(matrix) # We wanna know how many columns & rows in A matrix
        self.formulas = []  # This is a list of formulas to get the orthogonalized vectors
        self.formulas_2 = [] # This is a list of formulas to get the basis vectors
        self.computations = []   # This is a list of orthogonalized vectors that were computed
        self.e = []     # This is a list of computations made to get the basis vectors that will be used in computing orthogonalized vectors
        self.e_computations = [] # This is a list of basis vectors
        self.R_computations = []  # This is a list of computations made for the R matrix
    
    def GET_MATRIX_ROWS_COLUMNS(self, matrix):
        # Number of rows and columns in the matrix
        num_rows, num_cols = matrix.shape
        return num_rows, num_cols
    
    def DOT_PRODUCT(self, matrix_1, matrix_2):
        # Function for Getting dot product of two matrices
        result = np.dot(matrix_1, matrix_2)
        return result
    
    def MATRIX_PROJECTION(self, matrix_1, matrix_2):
        # Get projection of matrix to another matrix
        dot_product = self.DOT_PRODUCT(matrix_1, matrix_2)
        dot_product_itself = self.DOT_PRODUCT(matrix_2, matrix_2) # We need to call DOT_PRODUCT function for that

        # If in case there's an error
        if dot_product_itself == 0:
            raise ValueError("Matrix_2 has a dot product of zero with itself. Division by zero is not allowed.")

        projection = dot_product / dot_product_itself  # The formula to get the projection

        return projection  # Return the projection value
    
    def REARRANGE_MATRIX(self):     
        
        matrix = self.matrix
        
        # We wanna arrange the columns of A matrix first by arranging the columns in descending order according to their column norm
        for i in range(self.num_columns):
            max_norm = 0
            col_index = i
            
            for j in range(i, self.num_columns):
                u = self.matrix[:, j]
                norm_u = np.linalg.norm(u) # We use this to get the L2 norm of a column
                
                if norm_u > max_norm:
                    max_norm = norm_u
                    col_index = j
    
            # Swap the columns in the A matrix if the i^th column is not the column with the ^th largest norm
            if col_index != i:
                matrix[:, [i, col_index]] = matrix[:, [col_index, i]]
        print("          ")        
        print("Rearranged A matrix:")  
        print("          ")
        print(matrix)
        return matrix
        
    def compute_gram_schmidt(self):
        
        #Rearrange matrix columns to reduce chances of numerical instability
        self.matrix = self.REARRANGE_MATRIX()
        
        print("              ")
        print("Computations: ")
        print("              ")
        
        # We iterate over the columns and compute for u(i), e(i), a(i)*e(k) and we append them in the lists defined at the top
        for i in range(self.num_columns):
            formula = "u" + str(i) + " = a" + str(i) # The initial formula is just u(i) = a(i) but will expand if i > 0
            formula_2 = "e" + str(i) + " = u" + str(i) + " / norm(u" + str(i) + ")"
            u = self.matrix[:, i]
            e = u / np.linalg.norm(u) 
            self.e.append(e) # We append in the e list the computations of basis vector so we don't have to redo it later
            computation = u 
            
            for k in range(i+1): # This is for getting the a(i)*e(k) for the R matrix
                dot_product_scalar = self.MATRIX_PROJECTION(u, self.e[k]).astype(np.float64) # We call the project matrix function
                self.R_computations.append(dot_product_scalar) 
                
                print("I have computed a" + str(i) + "e" + str(k) + ": " + str(dot_product_scalar))
            
            for j in range(i - 1, -1, -1):
                added_formula = " - (a" + str(i) + " · e" + str(j) + ")e" + str(j)
                formula += added_formula # This is my solution for expanding the formula for every iteration in j
                dot_product_scalar = self.MATRIX_PROJECTION(u, self.e[j]).astype(np.float64) # This is the formula to get the projection of basis vector e to original vector a 
                computation = computation.astype(np.float64) - dot_product_scalar * self.e[j] # Continuous solving for u until j reaches -1

            self.formulas.append(formula)   # Here is where we append all the formulas and computations and after that we iterate over the next column
            self.formulas_2.append(formula_2)
            self.computations.append(computation)
            self.e_computations.append(e)  

    def display_formula_and_computations(self):
        for i in range(len(self.formulas)):
            formula = self.formulas[i]    # We want to display all the formulas and computations and put them in a list
            formula_2 = self.formulas_2[i]
            e_computation = self.e_computations[i]
            computation = self.computations[i]
            
            print(formula)  #Here is the printing part of the lists
            print(formula_2)
            print("e" + str(i) + " = " + str(e_computation))
            print("u" + str(i) + " = " + str(computation))
            
    def create_Q_matrix(self):
        # Convert the list of vectors into a matrix with column vectors
        column_vectors = [vector[:, np.newaxis] for vector in self.e_computations]
        matrix = np.hstack(column_vectors)
        return matrix
    
    def create_R_matrix(self):
        # Create first a R matrix full of zeroes
        R_matrix = np.zeros((self.num_rows, self.num_columns))
        
        # Then we iteratively put the R computations from the R_computations list to thw R matrix forming an upper eheclon matrix
        j = 0
        for i in range(self.num_columns):
            for k in range(i+1):
                R_matrix[k,i] = self.R_computations[j]
                j += 1
        
        return R_matrix          

In [9]:
#We use np.random to generate a random matrix
A_matrix = np.random.rand(6, 6)

print("A matrix =")
print("          ")
print(A_matrix)

gram_schmidt = Gram_Schmidth_Process(A_matrix) # We call the Gram Schmidt class first
gram_schmidt.compute_gram_schmidt() # We ask to make the computations
gram_schmidt.display_formula_and_computations() # Then we ask it to display the computations

Q_matrix = gram_schmidt.create_Q_matrix()  
print("          ")
print("Q matrix =") # Display the Q matrix
print("          ")
print(Q_matrix)

R_matrix = gram_schmidt.create_R_matrix()
print("          ")
print("R matrix =") # Display the R matrix
print("          ")
print(R_matrix)

A matrix =
          
[[0.76296529 0.5886862  0.82056856 0.99299435 0.30636578 0.79441055]
 [0.05980106 0.28161497 0.118322   0.95307974 0.7750838  0.75882719]
 [0.96252442 0.01082998 0.59019845 0.11971262 0.37706869 0.05861807]
 [0.573542   0.37173568 0.05017713 0.6406933  0.51606163 0.20248412]
 [0.14947201 0.01911447 0.70842407 0.88301279 0.0372132  0.38833504]
 [0.20786008 0.03181791 0.31867233 0.14004506 0.5293771  0.93140795]]
          
Rearranged A matrix:
          
[[0.99299435 0.79441055 0.76296529 0.82056856 0.30636578 0.5886862 ]
 [0.95307974 0.75882719 0.05980106 0.118322   0.7750838  0.28161497]
 [0.11971262 0.05861807 0.96252442 0.59019845 0.37706869 0.01082998]
 [0.6406933  0.20248412 0.573542   0.05017713 0.51606163 0.37173568]
 [0.88301279 0.38833504 0.14947201 0.70842407 0.0372132  0.01911447]
 [0.14004506 0.93140795 0.20786008 0.31867233 0.5293771  0.03181791]]
              
Computations: 
              
I have computed a0e0: 1.7659394093897243
I have computed a1e